In [ ]:
pip install python-dotenv

In [ ]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

BASE_URL = os.getenv('BASE_URL')
# BASE_URL = 'https://urtrade.trade'

def read_token(file_path='./token.txt'):
    try:
        with open(file_path, 'r') as file:
            return file.read().strip()
    except IOError as err:
        print(f'Error reading token from file: {err}')
        return None
    

def get_headers(token):
    return {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

def send_request(method, endpoint, params=None, json_data=None, quiet=False):
    """
    統一的 HTTP 請求處理核心
    :param method: "GET", "POST", "PUT", "DELETE" 等
    :param endpoint: API 路徑
    :param params: GET 請求的 Query Parameters
    :param json_data: POST/PUT 請求的 Body 資料
    """
    # 確保 URL 拼接邏輯一致 (處理可能多餘或缺少的斜線)
    url = f"{BASE_URL.rstrip('/')}/{endpoint.lstrip('/')}"
    
    try:
        token = read_token()
        # 使用 requests.request 動態決定方法
        response = requests.request(
            method=method,
            url=url,
            headers=get_headers(token),
            params=params,
            json=json_data
        )
        
        # 不raise，無論錯誤與否，後端會傳格式化json
        # response.raise_for_status()
        result = response.json()
        
        if not quiet:
            print(f"[{method}] {endpoint} Success: {result}")
            
        return result.get('data', [])

    except requests.exceptions.RequestException as err:
        print(f"Request failed ({method} {endpoint}): {err}")
        return None

In [ ]:
import datetime

def write_token(token):
    """將 Token 寫入檔案（'w' 模式會自動處理檔案不存在的情況）"""
    with open('token.txt', 'w', encoding='utf-8') as f:
        f.write(token)

# 2. 封裝重複的登入/註冊後處理邏輯 (DRY)
def _handle_auth_response(res):
    if res:
        token = res.get("token" ,'')
        write_token(token)
    return token

# 3. 業務邏輯函數
def login(password, email, quiet=False):
    res = send_request('POST', '/user/login', json_data={
        'email': email,
        'password': password,
    }, quiet=quiet)
    return _handle_auth_response(res)

def register(name, password, email, quiet=False):
    res = send_request('POST', '/user/register', json_data={
        'name': name,
        'password': password,
        'email': email,
    }, quiet=quiet)
    return _handle_auth_response(res)


def create_trade(company_id, trade_type, quantity, price):
    """
    新增單筆交易 (POST /trades)
    """
    endpoint = "/trades"
    today_str = datetime.date.today().isoformat()
    payload = {
        "companyId": company_id,
        "tradeType": trade_type,  # 'buy' or 'sell'
        "quantity": quantity,
        "price": price,
        "tradeDate": today_str
    }
    
    return send_request("POST", endpoint, json_data=payload)


def create_bulk_trades(trades_list):
    """
    新增多筆交易 (POST /trades/bulk)
    :param trades_list: 包含多個交易字典的列表
           例如: [{"companyId": 1, "tradeType": "buy", "quantity": 10, "price": 150.5}, ...]
    """
    endpoint = "/trades/bulk"
    payload = trades_list
    
    return send_request("POST", endpoint, json_data=payload)

def get_stock_symbols():
    """取得所有公司代號與名稱"""
    return send_request('GET', '/stock/symbols')

def get_today_stocks():
    """取得今日最新股票價格與漲跌數據"""
    return send_request('GET', '/stock/today')

def get_market_breadth():
    """取得今日市場漲跌寬度 (百分比)"""
    return send_request('GET', '/stock/breadth')



# 新聞api
def get_all_news(page=1, size=10, status='published'):
    """GET /news"""
    params = {"page": page, "size": size, "status": status}
    return send_request("GET", "/news", params=params)

def get_news_by_id(news_id):
    """GET /news/:id"""
    return send_request("GET", f"/news/{news_id}")

def create_news(content, status='draft', is_top=False):
    """POST /news"""
    payload = {
        "content": content,
        "status": status,
        "isTop": is_top
    }
    return send_request("POST", "/news", json_data=payload)

def bulk_create_news(news_list):
    """POST /news/bulkCreate"""
    # news_list 應該是一個 dict 的 list
    return send_request("POST", "/news/bulkCreate", json_data=news_list)

def update_news(news_id, update_data):
    """PUT /news/:id"""
    return send_request("PUT", f"/news/{news_id}", json_data=update_data)

def delete_news(news_id):
    """DELETE /news/:id"""
    return send_request("DELETE", f"/news/{news_id}")



# 市場動能
def get_all_momentum():
    return send_request('GET', 'market/momentum')

def get_momentum_by_range(days):
    return send_request('GET', f'market/momentum/range/{days}')
    
def get_market_weights():
    return send_request('GET', 'market/weights')

def get_last_asset_price(symbol):
    return send_request('GET', f'market/last/{symbol}')

def get_qutes():
    return send_request('GET', 'market/quotes')

def get_market_data_by_symbol(symbol, page=1, size=10):
    params = {'page': page, 'size': size}
    return send_request('GET', f'market/{symbol}', params=params)



# 餘額
def get_balance():
    """
    測試 取得餘額 (GET)
    """
    return send_request("GET", '/balance')

def create_balance(balance: float):
    """
    測試 初始化錢包 (POST)
    """
    json_data = {
        "balance": balance
    }
    return send_request("POST", '/balance', json_data=json_data)

def update_balance(target_balance: float):
    """
    測試 更新餘額至目標值 (PUT)
    對應你剛才寫的絕對值更新邏輯
    """
    json_data = {
        "balance": target_balance,
    }
    return send_request("PUT", '/balance', json_data=json_data)


# 業務邏輯層：只需關注 endpoint 和資料結構
def get_all_users(quiet=False):
    """取得所有使用者"""
    return send_request('GET', '/admin/users', quiet=quiet)

def set_user_as_admin(userId, quiet=False):
    """將使用者設為管理員"""
    res = send_request('POST', '/admin/set-admin', json_data={'userId': userId}, quiet=quiet)
    # 使用 getattr 或判斷 res 是否為 None，避免 res 為 None 時報錯
    return res.get('data') if res else None

def remove_admin(userId, quiet=False):
    """移除管理員權限"""
    return send_request('DELETE', f'/admin/admin/{userId}', quiet=quiet)

def remove_user(userId, quiet=False):
    """刪除使用者"""
    return send_request('DELETE', f'/admin/user/{userId}', quiet=quiet)


In [ ]:
import requests
import os

def create_trades_via_screenshot(file_path):
    """
    專門處理圖片分析上傳
    """
    endpoint = "/trade/analyze-screenshot"
    url = f"{BASE_URL.rstrip('/')}/{endpoint.lstrip('/')}"
    
    try:
        token = read_token()
        headers = get_headers(token)
        
        # 關鍵：上傳檔案時，必須讓 requests 自動產生 Content-Type (包含 boundary)
        # 'application/json'，一定要刪掉
        headers.pop('Content-Type', None)

        # 2. 準備檔案資料 (使用二進位模式 'rb')
        if not os.path.exists(file_path):
            return f"找不到檔案：{file_path}"

        filename = os.path.basename(file_path)
        
        with open(file_path, 'rb') as f:
            # key 必須是 'image'，對應你後端的 memUpload.single('image')
            files = {
                'image': (filename, f, 'image/jpeg')
            }
            
            print(f"正在傳送圖片至 AI 分析: {filename}...")
            
            response = requests.post(
                url=url,
                headers=headers,
                files=files,
                timeout=60  # 給 AI 一點反應時間
            )
        return response.json()

    except Exception as e:
        return f"發生錯誤: {str(e)}"

def get_ai_trade_extraction_status(jobId):
    return send_request('GET', f'/trade/ai-job/{jobId}')


In [ ]:
# --- 正確的圖片測試 ---
# 記得加上 r 避免 Unicode 錯誤
test_path = r'C:\Users\User\Downloads\IMG_20260210_194846.jpg'
result = create_trades_via_screenshot(test_path)
print(result)

In [ ]:
# 用無相干的圖測試
test_path = r'C:\Users\User\Downloads\3161772d4a8259d6c15bb2eea314fc76.webp'
result = create_trades_via_screenshot(test_path)
print(result)

In [ ]:
get_ai_trade_extraction_status('23fd0419-542a-40ec-b655-08d7b0930a65')

In [ ]:
register(name='', password='', email='')

In [ ]:
password=os.getenv('USER_PASSWORD')
email=os.getenv('USER_EMAIL')
login(password=password, email=email)

In [ ]:
create_trade(
    company_id=2,
    trade_type="sell",
    quantity=5,
    price=10
)

In [ ]:
today_str = datetime.date.today().isoformat()
create_bulk_trades([
    {
        "companyId": 1,
        "tradeType": "buy",
        "quantity": 10,
        "price": 50.00,
        "tradeDate": today_str
    },
    {
        "companyId": 2,
        "tradeType": "buy",
        "quantity": 10,
        "price": 2,
        "tradeDate": today_str
    }
])

In [ ]:
breadth = get_market_breadth()
print(f"目前市場寬度: {breadth * 100:.2f}%")

# 2. 取得今日股票列表並進行簡單處理
today_stocks = get_today_stocks()
if today_stocks:
    for stock in today_stocks:
        print(f"{stock['symbol']}: {stock['price']}")

In [ ]:
print("--- 測試創建新聞 ---")
create_res = create_news("這是測試內容", status="draft")

news_id = create_res.get('data', {}).get('id')

if news_id:
    # 2. 測試更新內容 (確認 MD5 也會跟著動)
    print("\n--- 測試更新新聞 ---")
    update_news(news_id, {"content": "內容更新了", "status": "published"})

    # 3. 測試讀取單一資料
    print("\n--- 測試讀取單一新聞 ---")
    get_news_by_id(news_id)

    # 4. 測試刪除
    print("\n--- 測試刪除新聞 ---")
    delete_news(news_id)

# 5. 測試批量創建
print("\n--- 測試批量創建 ---")
batch_payload = [
    {"content": "第一篇內容", "status": "published"},
    {"content": "第二篇內容", "status": "draft"}
]
bulk_create_news(batch_payload)

In [ ]:
get_all_momentum()

In [ ]:
get_last_asset_price("USOIL")

In [ ]:
get_qutes()

In [ ]:
get_market_data_by_symbol("BTCUSD")

In [ ]:
# create_balance(10000.00)
# get_balance()
update_balance(10000.0)

In [ ]:
get_momentum_by_range(1)

In [ ]:
get_momentum_by_range(7)

In [ ]:
bulk_create_news([
    {
        "content": "特斯拉正式宣布 Cybercab（Robotaxi）將於 2026 年 4 月進入全面量產階段。該車型採用蝶翼門設計且不設方向盤，搭配 FSD v13 的純視覺方案，標誌著特斯拉從汽車製造商轉型為全球領先的自動駕駛出行服務商。",
        "contentEn": "Tesla officially announced that Cybercab (Robotaxi) will enter full production in April 2026. Featuring butterfly doors and no steering wheel, powered by FSD v13's vision-only system, this marks Tesla's transition from an automaker to a global leader in autonomous mobility services.",
        "status": "published",
    }
])